In [66]:
import pandas as pd
import plotly.express as px
from sklearn.metrics import root_mean_squared_error, r2_score

In [67]:

pdf_output_header = pd.read_csv("data/output/header_output.csv")
pdf_output_detail = pd.read_csv("data/output/detail_output.csv")

In [68]:
pdf_output = pdf_output_header.merge(pdf_output_detail, on="model_output_id")

In [69]:
px.scatter(pdf_output, x="predicted_value", y="actual_value", color="model_type", hover_data=["name", "features_used"])

In [72]:

def evaluate_model_groups(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate R² and RMSE grouped by model_run_date, model_type, and version.

    Parameters:
    df (pd.DataFrame): DataFrame with actual and predicted values.

    Returns:
    pd.DataFrame: Grouped metrics.
    """
    grouped = df.groupby(["model_run_date", "model_type", "version", "features_used"])

    results = []
    for group_keys, group_df in grouped:
        r2 = r2_score(group_df["actual_value"], group_df["predicted_value"])
        rmse = root_mean_squared_error(group_df["actual_value"], group_df["predicted_value"])

        results.append({
            "model_run_date": group_keys[0],
            "model_type": group_keys[1],
            "version": group_keys[2],
            "features_used": group_keys[3],
            "feature_count": len(group_keys[3].split(",")),
            "r2_score": r2,
            "rmse": rmse
        })

    return pd.DataFrame(results).sort_values("r2_score", ascending=False)


In [73]:
evaluate_model_groups(pdf_output)

,model_run_date,model_type,version,features_used,feature_count,r2_score,rmse
5,2025-07-21 16:24:31.729886,XG_Boost,v0.0.1,"['value_last_year', 'age_last_year', 'pos_Atta...",19,0.818193,6.772602
4,2025-07-21 16:20:09.536323,XG_Boost,v0.0.1,"['value_last_year', 'age_last_year', 'pos_Atta...",6,0.806796,6.981645
7,2025-07-21 16:25:03.128236,Regression,v0.0.1,"['value_last_year', 'age_from_peak', 'pos_Atta...",19,0.804577,7.021623
6,2025-07-21 16:24:46.388963,Regression,v0.0.1,"['value_last_year', 'age_from_peak', 'pos_Atta...",6,0.804517,7.022709
1,2025-07-21 16:10:40.977813,Regression,v0.0.1,"['value_last_year', 'age_from_peak']",2,0.804191,7.028564
0,2025-07-21 16:10:36.871603,XG_Boost,v0.0.1,"['value_last_year', 'age_last_year']",2,0.802335,7.061792
3,2025-07-21 16:19:55.821087,Regression,v0.0.1,['value_last_year'],1,0.801800,7.071351
2,2025-07-21 16:19:40.243529,XG_Boost,v0.0.1,['value_last_year'],1,0.796712,7.161533
